In [4]:
using JuMP, Cbc

# model
m = Model()

#decision variables
@variable(m, 0 <= x[1:5] <= 1)  # x[i] is the fraction invested in project i

# NPV coefficients for each project
NPVs = [6, 5, 12, 12, 9]  

# cost coefficients for each project and year
costs = [
    [6, 2, 8, 10, 2],  # Costs for year 1
    [2, 5, 9, 11, 9],  # Costs for year 2
    [4, 3, 10, 9, 6],  # Costs for year 3
    [2, 3, 4, 8, 3]    # Costs for year 4
]

# budget for each year
annual_budget = 20   # Replace with the actual annual budget value

# Define the objective function (Maximize NPV)
@objective(m, Max, sum(NPVs[i] * x[i] for i in 1:5))

# budget constraints for each year

for t in 1:length(costs)
    @constraint(m, sum(costs[t][i] * x[i] for i in 1:5) <= annual_budget)
end

#costs[t][i] gets cost of each project in year t
# x[i] gets fraction of the investment allocated to project x 
#calculates the total investment cost for year t across all projects, weighted by the fraction invested in each project. 

#constraint for years 3 and 4 
for t in 1:length(costs)
    @constraint(m, sum(costs[t][i] * x[i] for i in 3:4) <= 0.4 * sum(costs[t][i] * x[i] for i in 1:5))
end

# Investment fraction constraint
@constraint(m, sum(x[i] for i in 1:5) == 1)

set_optimizer(m, Cbc.Optimizer)

# Solve the model
optimize!(m)

# Output the solution
println("Optimal investment fractions:")
for i in 1:5
    println("Project $i: ", value(x[i]))  # Corrected from value() to getvalue()
end
println("Maximized NPV: \$", objective_value(m), " million dollars")  # Corrected from objective_value() to getobjectivevalue()


Optimal investment fractions:
Project 1: 0.0
Project 2: 0.0
Project 3: 0.14285714285714288
Project 4: 0.0
Project 5: 0.8571428571428572
Maximized NPV: $9.42857142857143 million dollars
Presolve 8 (-1) rows, 5 (0) columns and 40 (-5) elements
0  Obj -0 Primal inf 1.0282846 (1) Dual inf 23.329274 (5)
5  Obj 9.4285714
Optimal - objective value 9.4285714
After Postsolve, objective 9.4285714, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9.428571429 - 5 iterations time 0.002, Presolve 0.00
